<a href="https://colab.research.google.com/github/varaprasad7006/SatyaVaraPrasad_INFO5731_Spring2025024/blob/main/vara_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[colab link ](https://colab.research.google.com/drive/1Io0Gl6Wym0TFKOPBst0BISFNE6RUQfBE?usp=sharing)

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Monday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (25 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]


(3) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(4) Collect all the information of the 904 narrators in the Densho Digital Repository.

(5)**Collect a total of 10000 reviews** of the top 100 most popular software from G2 and Capterra.


In [45]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import random

# Set Chrome options
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run in headless mode
chrome_options.add_argument('--no-sandbox')  # Bypass OS security model
chrome_options.add_argument('--disable-dev-shm-usage')  # Overcome limited resources

# Set user-agent headers
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

# Manually specify ChromeDriver path to avoid errors
service = Service("/usr/bin/chromedriver")
wd = webdriver.Chrome(service=service, options=chrome_options)

# Introduce a random delay to avoid detection
time.sleep(random.uniform(2, 5))




First review:
Best movie ever. Heath ledger's work is phenomenal no words......


In [19]:
pip install webdriver_manager

In [46]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Configure Chrome options to run Selenium in headless mode
chrome_options = Options()
chrome_options.add_argument('--headless')  # Ensures the browser runs in the background
chrome_options.add_argument('--no-sandbox')  # Bypasses OS security restrictions
chrome_options.add_argument('--disable-dev-shm-usage')  # Helps prevent crashes in limited resource environments
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")
chrome_options.add_argument('--blink-settings=imagesEnabled=false')  # Disables image loading to speed up execution

# Initialize the WebDriver using ChromeDriverManager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Open the IMDb reviews page for "The Dark Knight"
driver.get("https://www.imdb.com/title/tt0468569/reviews/")

# Scroll to the bottom to ensure the "All" button is visible, if present
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)  # Short delay to allow elements to load

# Attempt to locate and click the "All" button to display all reviews
try:
    all_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button//span[text()='All']"))
    )
    driver.execute_script("arguments[0].click();", all_button)
    time.sleep(5)  # Wait for additional reviews to load
except Exception as e:
    print(f"Could not click 'All' button: {e}")

# List to store extracted review text
reviews_list = []
max_reviews = 1000  # Define the maximum number of reviews to collect
batch_size = 25  # IMDb loads reviews in batches of 25
wait_time = 5  # Time delay for elements to load properly
retry_attempts = 3  # Maximum retries for "Load more reviews" button

while len(reviews_list) < max_reviews:
    # Locate review elements on the page
    reviews = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[@class='ipc-html-content-inner-div']"))
    )

    # Extract and store reviews in the list, ensuring no duplicates
    for review in reviews:
        if len(reviews_list) >= max_reviews:
            break
        review_text = review.text.strip()
        if review_text not in reviews_list:  # Avoid duplicate reviews
            reviews_list.append(review_text)
            print(f"Review {len(reviews_list)} collected: {review_text[:50]}...")  # Display a snippet of each review

    # Check if more reviews need to be loaded
    if len(reviews_list) < max_reviews:
        attempts = 0
        while attempts < retry_attempts:
            try:
                # Scroll to the bottom to trigger "Load more reviews" button
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)  # Wait for button to appear

                # Click the "Load more reviews" button (if available)
                load_more_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, "//button//span[contains(text(), '25 more')]")
                ))
                driver.execute_script("arguments[0].click();", load_more_button)
                time.sleep(wait_time)  # Allow new reviews to load
                break  # Exit retry loop upon success
            except Exception as e:
                print(f"Retry {attempts + 1}: Error clicking 'Load more' button: {e}")
                attempts += 1
                time.sleep(5)  # Wait before retrying
        else:
            print("Could not load additional reviews. Stopping data collection.")
            break

# Close the browser instance
driver.quit()

# Convert collected reviews into a DataFrame and save as CSV
reviews_df = pd.DataFrame(reviews_list, columns=["Review Text"])
reviews_df.to_csv('imdb_reviews.csv', index=False)

print(f"Reviews scraped: {len(reviews_data)}. Saved to 'imdb_reviews.csv'.")

Collected review 1: Best movie ever. Heath ledger's work is phenomenal...
Collected review 2: This movie is a work of art. The finest sequel eve...
Collected review 3: It is just what you want for the best movie. Great...
Collected review 4: Confidently directed, dark, brooding, and packed w...
Collected review 5: We've been subjected to enormous amounts of hype a...
Collected review 6: I couldn't believe "The Dark knight" could live up...
Collected review 7: Dark, yes, complex, ambitious. Christopher Nolan a...
Collected review 8: Im just gonna start off by saying I LOVE this movi...
Collected review 9: If someone else acted as Joker, I would give the m...
Collected review 10: I had the pleasure to watch this movie in an IMAX ...
Collected review 11: Do you REALLY think this was a better film than th...
Collected review 12: Christopher Nolan's second bundle of joy "The Dark...
Collected review 13: I saw the dark knight about three years ago when I...
Collected review 14: I used to lea

In [47]:
df

,review
0,Best movie ever. Heath ledger's work is phenom...
1,This movie is a work of art. The finest sequel...
2,It is just what you want for the best movie. G...
3,"Confidently directed, dark, brooding, and pack..."
4,We've been subjected to enormous amounts of hy...
...,...
995,"Acting, script, music, action, and cinematogra..."
996,This movie is great but not one of the best ev...
997,When I walked out of the movie I was asked wha...
998,"My wife and myself, parents of three little ki..."


# Question 2 (15 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [23]:
# Save the file to your Google Drive
df.to_csv('imdb_reviews.csv', index=False)

In [48]:
dataframe = pd.read_csv('imdb_reviews.csv')

In [49]:
dataframe.head()

,review
0,Best movie ever. Heath ledger's work is phenom...
1,This movie is a work of art. The finest sequel...
2,It is just what you want for the best movie. G...
3,"Confidently directed, dark, brooding, and pack..."
4,We've been subjected to enormous amounts of hy...


In [62]:
# Write code for each of the sub parts with proper comments.
#(1) Remove noise, such as special characters and punctuations.

import pandas as pd
import re

df_cleaned = dataframe.copy()

def remove_noise(feature):
  return re.sub(r'[^\w\s]','', feature)

df_cleaned["Cleaned_text"] = df_cleaned['review'].apply(remove_noise)

print(df_cleaned)


                                                review  \
0    Best movie ever. Heath ledger's work is phenom...   
1    This movie is a work of art. The finest sequel...   
2    It is just what you want for the best movie. G...   
3    Confidently directed, dark, brooding, and pack...   
4    We've been subjected to enormous amounts of hy...   
..                                                 ...   
995  Acting, script, music, action, and cinematogra...   
996  This movie is great but not one of the best ev...   
997  When I walked out of the movie I was asked wha...   
998  My wife and myself, parents of three little ki...   
999  Watching the various Batman incarnations over ...   

                                          Cleaned_text  
0    Best movie ever Heath ledgers work is phenomen...  
1    This movie is a work of art The finest sequel ...  
2    It is just what you want for the best movie Gr...  
3    Confidently directed dark brooding and packed ...  
4    Weve been sub

In [63]:
df_cleaned

,review,Cleaned_text
0,Best movie ever. Heath ledger's work is phenom...,Best movie ever Heath ledgers work is phenomen...
1,This movie is a work of art. The finest sequel...,This movie is a work of art The finest sequel ...
2,It is just what you want for the best movie. G...,It is just what you want for the best movie Gr...
3,"Confidently directed, dark, brooding, and pack...",Confidently directed dark brooding and packed ...
4,We've been subjected to enormous amounts of hy...,Weve been subjected to enormous amounts of hyp...
...,...,...
995,"Acting, script, music, action, and cinematogra...",Acting script music action and cinematography ...
996,This movie is great but not one of the best ev...,This movie is great but not one of the best ev...
997,When I walked out of the movie I was asked wha...,When I walked out of the movie I was asked wha...
998,"My wife and myself, parents of three little ki...",My wife and myself parents of three little kid...


In [64]:
#2 Remove numbers.
import re

def remove_numbers(feature):
    feature = re.sub(r'\d+', '', feature)

df_cleaned["Cleaned_text"] = df_cleaned['Cleaned_text'].apply(lambda x: re.sub(r'\d+', '', x))

print(df_cleaned.head())


                                              review  \
0  Best movie ever. Heath ledger's work is phenom...   
1  This movie is a work of art. The finest sequel...   
2  It is just what you want for the best movie. G...   
3  Confidently directed, dark, brooding, and pack...   
4  We've been subjected to enormous amounts of hy...   

                                        Cleaned_text  
0  Best movie ever Heath ledgers work is phenomen...  
1  This movie is a work of art The finest sequel ...  
2  It is just what you want for the best movie Gr...  
3  Confidently directed dark brooding and packed ...  
4  Weve been subjected to enormous amounts of hyp...  


In [67]:
print(df_cleaned.iloc[0,0])
print(df_cleaned.iloc[0,1])

Best movie ever. Heath ledger's work is phenomenal no words......
Best movie ever Heath ledgers work is phenomenal no words


In [70]:
#(3) Remove stopwords by using the stopwords list.
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))  # Load English stopwords

def remove_stop_words(feature):
    feature = ' '.join([word for word in feature.split() if word not in stop_words])

df_cleaned["Cleaned_text"] = df_cleaned['Cleaned_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

print(df_cleaned)


                                                review  \
0    Best movie ever. Heath ledger's work is phenom...   
1    This movie is a work of art. The finest sequel...   
2    It is just what you want for the best movie. G...   
3    Confidently directed, dark, brooding, and pack...   
4    We've been subjected to enormous amounts of hy...   
..                                                 ...   
995  Acting, script, music, action, and cinematogra...   
996  This movie is great but not one of the best ev...   
997  When I walked out of the movie I was asked wha...   
998  My wife and myself, parents of three little ki...   
999  Watching the various Batman incarnations over ...   

                                          Cleaned_text  
0    Best movie ever Heath ledgers work phenomenal ...  
1    This movie work art The finest sequel ever mad...  
2    It want best movie Great story great acting th...  
3    Confidently directed dark brooding packed impr...  
4    Weve subjecte

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [71]:
df_cleaned.iloc[0,1]

'Best movie ever Heath ledgers work phenomenal words'

In [72]:
#(4) Lowercase all texts
def lowercase(feature):
    feature = feature.lower()

df_cleaned["Cleaned_text"] = df_cleaned['Cleaned_text'].apply(lowercase)

print(df_cleaned)


                                                review Cleaned_text
0    Best movie ever. Heath ledger's work is phenom...         None
1    This movie is a work of art. The finest sequel...         None
2    It is just what you want for the best movie. G...         None
3    Confidently directed, dark, brooding, and pack...         None
4    We've been subjected to enormous amounts of hy...         None
..                                                 ...          ...
995  Acting, script, music, action, and cinematogra...         None
996  This movie is great but not one of the best ev...         None
997  When I walked out of the movie I was asked wha...         None
998  My wife and myself, parents of three little ki...         None
999  Watching the various Batman incarnations over ...         None

[1000 rows x 2 columns]


In [139]:
#(5) Stemming.
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

def stems(features):
    words = features.split()
    return ' '.join([stemmer.stem(word) for word in words])




In [89]:
df_cleaned

,review,Cleaned_text
0,Best movie ever. Heath ledger's work is phenom...,None
1,This movie is a work of art. The finest sequel...,None
2,It is just what you want for the best movie. G...,None
3,"Confidently directed, dark, brooding, and pack...",None
4,We've been subjected to enormous amounts of hy...,None
...,...,...
995,"Acting, script, music, action, and cinematogra...",None
996,This movie is great but not one of the best ev...,None
997,When I walked out of the movie I was asked wha...,None
998,"My wife and myself, parents of three little ki...",None


In [91]:
#(6) Lemmatization.
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

lemmizer = WordNetLemmatizer()

def lems(feature):
  word = feature.split()
  return ' '.join([lemmizer.lemmatize(word) for word in word])

df_cleaned["Cleaned_text"] = df_cleaned['Cleaned_text'].apply(lems)

print(df_cleaned)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                                review  \
0    Best movie ever. Heath ledger's work is phenom...   
1    This movie is a work of art. The finest sequel...   
2    It is just what you want for the best movie. G...   
3    Confidently directed, dark, brooding, and pack...   
4    We've been subjected to enormous amounts of hy...   
..                                                 ...   
995  "The Dark Knight" isn't just a movie; it's a c...   
996  No one could create a unique Batman better tha...   
997  Quite simply "Dark Knight" is the best superhe...   
998  Batman (Christian Bale) joins force with Lieut...   
999  Writing this review at the tail end of opening...   

                                          Cleaned_text  
0    best movi ever heath ledger work is phenomen n...  
1    thi movi is a work of art the finest sequel ev...  
2    it is just what you want for the best movi gre...  
3    confid direct dark brood and pack with impress...  
4    weve been sub

In [92]:
df_cleaned.iloc[0,1]

'best movi ever heath ledger work is phenomen no word'

In [93]:
df_cleaned.to_csv('reviews_cleaned.csv', index=False)

In [43]:
df_cleaned

,review,Cleaned_text
0,Best movie ever. Heath ledger's work is phenom...,best movi ever heath ledger work phenomen word
1,This movie is a work of art. The finest sequel...,thi movi work art the finest sequel ever made ...
2,It is just what you want for the best movie. G...,it want best movi great stori great act thrill...
3,"Confidently directed, dark, brooding, and pack...",confid direct dark brood pack impress action s...
4,We've been subjected to enormous amounts of hy...,weve subject enorm amount hype market dark kni...
...,...,...
995,"""The Dark Knight"" isn't just a movie; it's a c...",the dark knight isnt movi cinemat success tran...
996,No one could create a unique Batman better tha...,no one could creat uniqu batman better nolan
997,"Quite simply ""Dark Knight"" is the best superhe...",quit simpli dark knight best superhero movi ev...
998,Batman (Christian Bale) joins force with Lieut...,batman christian bale join forc lieuten jame g...


# Question 3 (15 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [94]:
# Your code here

#1 Parts of Speech(POS)
import spacy

spacy_eng_model = spacy.load("en_core_web_sm")

total_counts = {
    "nouns": 0,
    "verbs": 0,
    "adjectives": 0,
    "adverbs": 0
}

def pos(feature):
    pos_model = spacy_eng_model(feature)  # main POS happening here
    counts = {
        "nouns": 0,
        "verbs": 0,
        "adjectives": 0,
        "adverbs": 0
    }

    for words in pos_model:
        if words.pos_ == "NOUN":
            counts["nouns"] += 1
        elif words.pos_ == "VERB":
            counts["verbs"] += 1
        elif words.pos_ == "ADJ":
            counts["adjectives"] += 1
        elif words.pos_ == "ADV":
            counts["adverbs"] += 1

    total_counts["nouns"] += counts["nouns"]
    total_counts["verbs"] += counts["verbs"]
    total_counts["adjectives"] += counts["adjectives"]
    total_counts["adverbs"] += counts["adverbs"]
    return f"nouns = {counts['nouns']}, verbs = {counts['verbs']}, adjectives = {counts['adjectives']}, adverbs = {counts['adverbs']}"


df_cleaned["POS_Counts"] = df_cleaned["Cleaned_text"].apply(pos)

In [95]:
# Print total POS counts
print("=== POS Tagging Counts ===")
print(f"Total Nouns: {total_counts['nouns']}")
print(f"Total Verbs: {total_counts['verbs']}")
print(f"Total Adjectives: {total_counts['adjectives']}")
print(f"Total Adverbs: {total_counts['adverbs']}")


=== POS Tagging Counts ===
Total Nouns: 43930
Total Verbs: 20467
Total Adjectives: 15762
Total Adverbs: 7913


In [96]:
df_cleaned.head()

,review,Cleaned_text,POS_Counts
0,Best movie ever. Heath ledger's work is phenom...,best movi ever heath ledger work is phenomen n...,"nouns = 4, verbs = 1, adjectives = 1, adverbs = 1"
1,This movie is a work of art. The finest sequel...,thi movi is a work of art the finest sequel ev...,"nouns = 18, verbs = 6, adjectives = 12, adverb..."
2,It is just what you want for the best movie. G...,it is just what you want for the best movi gre...,"nouns = 9, verbs = 5, adjectives = 5, adverbs = 4"
3,"Confidently directed, dark, brooding, and pack...",confid direct dark brood and pack with impress...,"nouns = 17, verbs = 5, adjectives = 6, adverbs..."
4,We've been subjected to enormous amounts of hy...,weve been subject to enorm amount of hype and ...,"nouns = 84, verbs = 29, adjectives = 21, adver..."


In [97]:
!pip install spacy benepar pandas
!python -m spacy download en_core_web_sm
!python -m benepar.download benepar_en3

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.8 MB/s eta 0:00:00
  Created wheel for benepar: filename=benepar-0.2.0-py3-none-any.whl size=37626 sha256=7dd6d56a0fbd73130398c6c87d75b04b9718181c5dcfbfdb72805281b0eb2322
  Stored in

In [99]:
#3 Named Entity Recognition
#Named Entity Recognition: Extract all the entities such as person names, organizations,
#locations, product names, and date from the clean texts, calculate the count of each entity.
import spacy

# Load the English language model from spaCy
spacy_eng_model = spacy.load("en_core_web_sm")

def ner(text):
    """Perform Named Entity Recognition (NER) on the given text and count occurrences of specific entity types."""
    ner_model = spacy_eng_model(text)

    # Dictionary to store entity counts
    entity_counts = {
        "person_names": 0,
        "organizations": 0,
        "locations": 0,
        "product_names": 0,
        "dates": 0
    }

    # Iterate over detected entities and categorize them
    for entity in ner_model.ents:
        if entity.label_ == "PERSON":
            entity_counts["person_names"] += 1
        elif entity.label_ == "ORG":
            entity_counts["organizations"] += 1
        elif entity.label_ == "LOC":
            entity_counts["locations"] += 1
        elif entity.label_ == "PRODUCT":
            entity_counts["product_names"] += 1
        elif entity.label_ == "DATE":
            entity_counts["dates"] += 1

    # Return entity counts in a formatted string
    return (f"person_names = {entity_counts['person_names']}, "
            f"organizations = {entity_counts['organizations']}, "
            f"locations = {entity_counts['locations']}, "
            f"product_names = {entity_counts['product_names']}, "
            f"dates = {entity_counts['dates']}")

# Apply the NER function to the "Cleaned_text" column and store the results in a new column
df_cleaned["NER_Counts"] = df_cleaned["Cleaned_text"].apply(ner)


In [100]:
df_cleaned

,review,Cleaned_text,POS_Counts,NER_Counts
0,Best movie ever. Heath ledger's work is phenom...,best movi ever heath ledger work is phenomen n...,"nouns = 4, verbs = 1, adjectives = 1, adverbs = 1","person_names = 0, organizations = 0, locations..."
1,This movie is a work of art. The finest sequel...,thi movi is a work of art the finest sequel ev...,"nouns = 18, verbs = 6, adjectives = 12, adverb...","person_names = 0, organizations = 0, locations..."
2,It is just what you want for the best movie. G...,it is just what you want for the best movi gre...,"nouns = 9, verbs = 5, adjectives = 5, adverbs = 4","person_names = 0, organizations = 0, locations..."
3,"Confidently directed, dark, brooding, and pack...",confid direct dark brood and pack with impress...,"nouns = 17, verbs = 5, adjectives = 6, adverbs...","person_names = 0, organizations = 0, locations..."
4,We've been subjected to enormous amounts of hy...,weve been subject to enorm amount of hype and ...,"nouns = 84, verbs = 29, adjectives = 21, adver...","person_names = 4, organizations = 0, locations..."
...,...,...,...,...
995,"""The Dark Knight"" isn't just a movie; it's a c...",the dark knight isnt just a movi it a cinemat ...,"nouns = 31, verbs = 4, adjectives = 10, adverb...","person_names = 1, organizations = 1, locations..."
996,No one could create a unique Batman better tha...,no one could creat a uniqu batman better than ...,"nouns = 2, verbs = 1, adjectives = 0, adverbs = 1","person_names = 0, organizations = 0, locations..."
997,"Quite simply ""Dark Knight"" is the best superhe...",quit simpli dark knight is the best superhero ...,"nouns = 85, verbs = 45, adjectives = 43, adver...","person_names = 1, organizations = 0, locations..."
998,Batman (Christian Bale) joins force with Lieut...,batman christian bale join forc with lieuten j...,"nouns = 76, verbs = 12, adjectives = 26, adver...","person_names = 5, organizations = 1, locations..."


# **Following Questions must answer using AI assitance**

#Question 4 (20 points).

Q4. (PART-1)
Web scraping data from the GitHub Marketplace to gather details about popular actions. Using Python, the process begins by sending HTTP requests to multiple pages of the marketplace (1000 products), handling pagination through dynamic page numbers. The key details extracted include the product name, a short description, and the URL.

 The extracted data is stored in a structured CSV format with columns for product name, description, URL, and page number. A time delay is introduced between requests to avoid server overload. ChatGPT can assist by helping with the parsing of HTML, error handling, and generating reports based on the data collected.

 The goal is to complete the scraping within a specified time limit, ensuring that the process is efficient and adheres to GitHub’s usage guidelines.

(PART -2)

1.   **Preprocess Data**: Clean the text by tokenizing, removing stopwords, and converting to lowercase.

2. Perform **Data Quality** operations.


Preprocessing:
Preprocessing involves cleaning the text by removing noise such as special characters, HTML tags, and unnecessary whitespace. It also includes tasks like tokenization, stopword removal, and lemmatization to standardize the text for analysis.

Data Quality:
Data quality checks ensure completeness, consistency, and accuracy by verifying that all required columns are filled and formatted correctly. Additionally, it involves identifying and removing duplicates, handling missing values, and ensuring the data reflects the true content accurately.


Github MarketPlace page:
https://github.com/marketplace?type=actions

In [102]:
import time
import random
time.sleep(random.uniform(3, 7))  # Random delay to avoid detection


In [103]:
import requests
import pandas as pd

# GitHub API endpoint for marketplace actions
url = "https://api.github.com/search/repositories?q=topic:github-actions&sort=stars&order=desc"

# GitHub API requires authentication for higher limits
headers = {
    "Accept": "application/vnd.github.v3+json",
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    actions = [[repo["name"], repo["description"], repo["html_url"]] for repo in data["items"]]

    df = pd.DataFrame(actions, columns=["Name", "Description", "URL"])
    df.to_csv("github_actions_api.csv", index=False, encoding="utf-8")

    print("✅ Data saved as github_actions_api.csv")
else:
    print(f"❌ API request failed. Status Code: {response.status_code}")


✅ Data saved as github_actions_api.csv


In [104]:
df2 = pd.read_csv("/content/github_actions_api.csv")

In [106]:
df2

,Name,Description,URL
0,act,Run your GitHub Actions locally 🚀,https://github.com/nektos/act
1,gitea,Git with a cup of tea! Painless self-hosted al...,https://github.com/go-gitea/gitea
2,awesome-actions,A curated list of awesome actions to use on Gi...,https://github.com/sdras/awesome-actions
3,goreleaser,"Release engineering, simplified",https://github.com/goreleaser/goreleaser
4,Actions-OpenWrt,A template for building OpenWrt with GitHub Ac...,https://github.com/P3TERX/Actions-OpenWrt
5,community,Public feedback discussions for: GitHub Mobile...,https://github.com/community/community
6,OpenWrt-Rpi,Raspberry Pi & NanoPi R2S/R4S & G-Dock & x86 O...,https://github.com/SuLingGG/OpenWrt-Rpi
7,danger-js,"⚠️ Stop saying ""you forgot to …"" in code review",https://github.com/danger/danger-js
8,ssh-action,GitHub Actions for executing remote ssh commands.,https://github.com/appleboy/ssh-action
9,actions-runner-controller,Kubernetes controller for GitHub Actions self-...,https://github.com/actions/actions-runner-cont...


In [108]:
df2.head(50)

,Name,Description,URL,Page Number
0,act,Run your GitHub Actions locally 🚀,https://github.com/nektos/act,1
1,gitea,Git with a cup of tea! Painless self-hosted al...,https://github.com/go-gitea/gitea,1
2,awesome-actions,A curated list of awesome actions to use on Gi...,https://github.com/sdras/awesome-actions,1
3,goreleaser,"Release engineering, simplified",https://github.com/goreleaser/goreleaser,1
4,Actions-OpenWrt,A template for building OpenWrt with GitHub Ac...,https://github.com/P3TERX/Actions-OpenWrt,1
5,community,Public feedback discussions for: GitHub Mobile...,https://github.com/community/community,1
6,OpenWrt-Rpi,Raspberry Pi & NanoPi R2S/R4S & G-Dock & x86 O...,https://github.com/SuLingGG/OpenWrt-Rpi,1
7,danger-js,"⚠️ Stop saying ""you forgot to …"" in code review",https://github.com/danger/danger-js,1
8,ssh-action,GitHub Actions for executing remote ssh commands.,https://github.com/appleboy/ssh-action,1
9,actions-runner-controller,Kubernetes controller for GitHub Actions self-...,https://github.com/actions/actions-runner-cont...,1


# Preprocess Data: Clean the text by tokenizing, removing stopwords, and converting to lowercase.

In [109]:
!pip install nltk

In [110]:
import nltk

# Download all required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')  # Explicitly download punkt_tab

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [113]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Load the CSV file into a DataFrame
file_path = 'github_marketplace_products.csv'
# df2 = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df2.head())

# Ensure the 'Page Number' column exists
if 'Page Number' not in df2.columns:
    df2['Page Number'] = (df2.index // 20) + 1

# Define the preprocessing function
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text)
    # Converting to Lowercase
    tokens = [word.lower() for word in tokens]
    # Removing Stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Join tokens back into a single string
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text

# Apply the preprocessing function to the text column
# Assuming the text column is named 'Description'
df2['cleaned_description'] = df2['Description'].apply(preprocess_text)

# Display the first few rows of the DataFrame with cleaned text and Page Number
print(df2[['Description', 'cleaned_description', 'Page Number']].head())

# Save the updated DataFrame back to the same CSV file
df2.to_csv(file_path, index=False)
print(f"Updated data saved to {file_path}")



              Name                                        Description  \
0              act                  Run your GitHub Actions locally 🚀   
1            gitea  Git with a cup of tea! Painless self-hosted al...   
2  awesome-actions  A curated list of awesome actions to use on Gi...   
3       goreleaser                    Release engineering, simplified   
4  Actions-OpenWrt  A template for building OpenWrt with GitHub Ac...   

                                         URL  Page Number  
0              https://github.com/nektos/act            1  
1          https://github.com/go-gitea/gitea            1  
2   https://github.com/sdras/awesome-actions            1  
3   https://github.com/goreleaser/goreleaser            1  
4  https://github.com/P3TERX/Actions-OpenWrt            1  
                                         Description  \
0                  Run your GitHub Actions locally 🚀   
1  Git with a cup of tea! Painless self-hosted al...   
2  A curated list of awesome acti

In [114]:
df2.head()

,Name,Description,URL,Page Number,cleaned_description
0,act,Run your GitHub Actions locally 🚀,https://github.com/nektos/act,1,run github actions locally 🚀
1,gitea,Git with a cup of tea! Painless self-hosted al...,https://github.com/go-gitea/gitea,1,git cup tea ! painless self-hosted all-in-one ...
2,awesome-actions,A curated list of awesome actions to use on Gi...,https://github.com/sdras/awesome-actions,1,curated list awesome actions use github
3,goreleaser,"Release engineering, simplified",https://github.com/goreleaser/goreleaser,1,"release engineering , simplified"
4,Actions-OpenWrt,A template for building OpenWrt with GitHub Ac...,https://github.com/P3TERX/Actions-OpenWrt,1,template building openwrt github actions | 使用 ...


In [115]:
df2.iloc[0,3]

1

# Data quality step

In [124]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from urllib.parse import urlparse

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Display initial rows of the DataFrame
print(df2.head())

# Ensure 'Page Number' column exists or create it based on row index
if 'Page Number' not in df2.columns:
    df2['Page Number'] = (df2.index // 20) + 1

# Function to clean and preprocess text
def preprocess_text(text):
    """Cleans and preprocesses the input text by removing HTML, special characters, stopwords, and applying lemmatization."""
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove non-alphanumeric characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove excess whitespace

    # Tokenization and lowercase conversion
    tokens = word_tokenize(text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    filtered_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    return ' '.join(filtered_tokens)  # Convert list back to string

# Apply preprocessing to the 'Description' column
df2['cleaned_description'] = df2['Description'].apply(preprocess_text)

# Data Quality Checks

# 1. Completeness: Remove rows with missing values in essential columns
required_columns = ['Name', 'URL', 'Description', 'Page Number']
df2.dropna(subset=required_columns, inplace=True)

# 2. Consistency: Ensure data types are correct
df2['Name'] = df2['Name'].astype(str)
df2['Description'] = df2['Description'].astype(str)

# Validate URLs
def is_valid_url(url):
    """Checks if a given URL is properly structured."""
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

# Remove rows with invalid URLs
df2 = df2[df2['URL'].apply(is_valid_url)]

# 3. Accuracy: Remove duplicate entries
df2.drop_duplicates(inplace=True)

# Display the cleaned DataFrame with relevant columns
print(df2[['Name', 'URL', 'Description', 'cleaned_description', 'Page Number']].head())

# Save the cleaned data to a new CSV file
df2.to_csv("new_git", index=False)


              Name                                        Description  \
0              act                  Run your GitHub Actions locally 🚀   
1            gitea  Git with a cup of tea! Painless self-hosted al...   
2  awesome-actions  A curated list of awesome actions to use on Gi...   
3       goreleaser                    Release engineering, simplified   
4  Actions-OpenWrt  A template for building OpenWrt with GitHub Ac...   

                                         URL  Page Number  \
0              https://github.com/nektos/act            1   
1          https://github.com/go-gitea/gitea            1   
2   https://github.com/sdras/awesome-actions            1   
3   https://github.com/goreleaser/goreleaser            1   
4  https://github.com/P3TERX/Actions-OpenWrt            1   

                                 cleaned_description  
0                          run github action locally  
1  git cup tea painless selfhosted allinone softw...  
2             curated list aw

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [125]:
df2.head()

,Name,Description,URL,Page Number,cleaned_description
0,act,Run your GitHub Actions locally 🚀,https://github.com/nektos/act,1,run github action locally
1,gitea,Git with a cup of tea! Painless self-hosted al...,https://github.com/go-gitea/gitea,1,git cup tea painless selfhosted allinone softw...
2,awesome-actions,A curated list of awesome actions to use on Gi...,https://github.com/sdras/awesome-actions,1,curated list awesome action use github
3,goreleaser,"Release engineering, simplified",https://github.com/goreleaser/goreleaser,1,release engineering simplified
4,Actions-OpenWrt,A template for building OpenWrt with GitHub Ac...,https://github.com/P3TERX/Actions-OpenWrt,1,template building openwrt github action github...


In [126]:
df2.isnull().sum()

,0
Name,0
Description,0
URL,0
Page Number,0
cleaned_description,0


In [127]:
df2.to_csv("github_marketplace_products_cleaned.csv", index=False)

#Question 5 (20 points)

PART 1:
Web Scrape  tweets from Twitter using the Tweepy API, specifically targeting hashtags related to subtopics (machine learning or artificial intelligence.)
The extracted data includes the tweet ID, username, and text.

Part 2:
Perform data cleaning procedures

A final data quality check ensures the completeness and consistency of the dataset. The cleaned data is then saved into a CSV file for further analysis.


**Note**

1.   Follow tutorials provided in canvas to obtain api keys. Use ChatGPT to get the code. Make sure the file is downloaded and saved.
2.   Make sure you divide GPT code as shown in tutorials, dont make multiple requestes.


In [128]:
import tweepy
import time
import pandas as pd

In [129]:
import tweepy
import pandas as pd

# 🔑 Use your own API credentials
bearer_token = "AAAAAAAAAAAAAAAAAAAAAN2fzQEAAAAArygzzPd5C4j76sPf1xN326UHD6o%3DOKhC9iJ31Ag63DY4OVKAR0ooUT3c3CJiQsGLTX6c1dyI0A1oeZ"

# ✅ Authenticate using Bearer Token (API v2)
client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

# Define query (use -is:retweet to remove retweets)
query = "Machine Learning OR Artificial Intelligence -is:retweet lang:en"

# Fetch recent tweets (max 100 per request)
tweets = client.search_recent_tweets(query=query, max_results=100, tweet_fields=["id", "text", "created_at", "author_id"])

# Convert to DataFrame
tweets_data = [[tweet.id, tweet.author_id, tweet.created_at, tweet.text] for tweet in tweets.data]
df = pd.DataFrame(tweets_data, columns=["Tweet_ID", "Author_ID", "Timestamp", "Tweet_Text"])

# Save to CSV
df.to_csv("twitter_ml_ai_tweets.csv", index=False, encoding="utf-8")

print(f"✅ Scraped {len(df)} tweets and saved to CSV!")


✅ Scraped 100 tweets and saved to CSV!


In [130]:
df

,Tweet_ID,Author_ID,Timestamp,Tweet_Text
0,1892389961896513618,16273831,2025-02-20 01:45:00+00:00,▶️A German study shows that the breast cancer ...
1,1892389915557576729,1892385091911110658,2025-02-20 01:44:49+00:00,WHAT IS ARTIFICIAL INTELLIGENCE?\n\nAI isn’t s...
2,1892389914462900504,73691428,2025-02-20 01:44:49+00:00,RT @7etsuo: This series consists of four artic...
3,1892389877972455647,1671555700374360064,2025-02-20 01:44:40+00:00,Discover the latest insights on artificial int...
4,1892389875665494212,1671555700374360064,2025-02-20 01:44:39+00:00,Explore the fascinating world of Artificial In...
...,...,...,...,...
95,1892386333261230325,906017180,2025-02-20 01:30:35+00:00,RT @7etsuo: This series consists of four artic...
96,1892386286737969186,23296198,2025-02-20 01:30:24+00:00,RT @7etsuo: This series consists of four artic...
97,1892386219134243177,92342510,2025-02-20 01:30:08+00:00,RT @GoogleCloudTech: Vertex AI RAG Engine is a...
98,1892386200788332649,29404618,2025-02-20 01:30:03+00:00,Discover the Surprising Techniques Behind Our ...


In [ ]:
from google.colab import drive


# Save DataFrame to Google Drive
df.to_csv(file_path, index=False)


In [ ]:
# df = pd.read_csv("/content/drive/MyDrive/tweets.csv")

In [ ]:
df.drop(["author_id","created_at"], axis =1 ,inplace = True)

In [ ]:
df.head()

# Part 2 Perform data cleaning procedures

In [131]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [132]:
df = pd.read_csv("twitter_ml_ai_tweets.csv")

In [134]:
df

,Tweet_ID,Author_ID,Timestamp,Tweet_Text
0,1892389961896513618,16273831,2025-02-20 01:45:00+00:00,▶️A German study shows that the breast cancer ...
1,1892389915557576729,1892385091911110658,2025-02-20 01:44:49+00:00,WHAT IS ARTIFICIAL INTELLIGENCE?\n\nAI isn’t s...
2,1892389914462900504,73691428,2025-02-20 01:44:49+00:00,RT @7etsuo: This series consists of four artic...
3,1892389877972455647,1671555700374360064,2025-02-20 01:44:40+00:00,Discover the latest insights on artificial int...
4,1892389875665494212,1671555700374360064,2025-02-20 01:44:39+00:00,Explore the fascinating world of Artificial In...
...,...,...,...,...
95,1892386333261230325,906017180,2025-02-20 01:30:35+00:00,RT @7etsuo: This series consists of four artic...
96,1892386286737969186,23296198,2025-02-20 01:30:24+00:00,RT @7etsuo: This series consists of four artic...
97,1892386219134243177,92342510,2025-02-20 01:30:08+00:00,RT @GoogleCloudTech: Vertex AI RAG Engine is a...
98,1892386200788332649,29404618,2025-02-20 01:30:03+00:00,Discover the Surprising Techniques Behind Our ...


In [135]:
def clean_tweet_text(text):
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    # Remove mentions (@username)
    text = re.sub(r"@\w+", "", text)
    # Remove special characters and numbers (keep only letters and spaces)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra spaces
    text = " ".join(text.split())
    return text

df["cleaned_tweets"] = df["Tweet_Text"].apply(clean_tweet_text)

In [136]:
df["cleaned_tweets"].iloc[2]

'rt this series consists of four articles part introduction to machine learning in c a beginnerfriendly overview of machine'

In [137]:

# Save DataFrame to Google Drive
df.to_csv("tweets_cleaned.csv", index=False)

In [138]:
df

,Tweet_ID,Author_ID,Timestamp,Tweet_Text,cleaned_tweets
0,1892389961896513618,16273831,2025-02-20 01:45:00+00:00,▶️A German study shows that the breast cancer ...,a german study shows that the breast cancer de...
1,1892389915557576729,1892385091911110658,2025-02-20 01:44:49+00:00,WHAT IS ARTIFICIAL INTELLIGENCE?\n\nAI isn’t s...,what is artificial intelligence ai isnt some s...
2,1892389914462900504,73691428,2025-02-20 01:44:49+00:00,RT @7etsuo: This series consists of four artic...,rt this series consists of four articles part ...
3,1892389877972455647,1671555700374360064,2025-02-20 01:44:40+00:00,Discover the latest insights on artificial int...,discover the latest insights on artificial int...
4,1892389875665494212,1671555700374360064,2025-02-20 01:44:39+00:00,Explore the fascinating world of Artificial In...,explore the fascinating world of artificial in...
...,...,...,...,...,...
95,1892386333261230325,906017180,2025-02-20 01:30:35+00:00,RT @7etsuo: This series consists of four artic...,rt this series consists of four articles part ...
96,1892386286737969186,23296198,2025-02-20 01:30:24+00:00,RT @7etsuo: This series consists of four artic...,rt this series consists of four articles part ...
97,1892386219134243177,92342510,2025-02-20 01:30:08+00:00,RT @GoogleCloudTech: Vertex AI RAG Engine is a...,rt vertex ai rag engine is a managed service t...
98,1892386200788332649,29404618,2025-02-20 01:30:03+00:00,Discover the Surprising Techniques Behind Our ...,discover the surprising techniques behind our ...


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

there are different ways in doing this assignment properly there are some ways whcih can be chalenging we can use so many libraries and apis but the ones that are recomended by specific url or website can be used for its good and alos for better and faseter process and for it took lots and lots of time to complete cause of i tested it with different methods available.

# Write your response below
Fill out survey and provide your valuable feedback.

https://docs.google.com/forms/d/e/1FAIpQLSd_ObuA3iNoL7Az_C-2NOfHodfKCfDzHZtGRfIker6WyZqTtA/viewform?usp=dialog